In [1]:
import tensorflow as tf
import numpy as np
import math

In [2]:
g1 = tf.Graph()

In [3]:
from easydict import EasyDict as edict
args = edict({'hidden_size':1, 'batch_size':1, 'output_size': 2, 'seq_len': 3, 'train_method': "single", 'max_grad_norm':5.0, 'grad_clip': True})

In [4]:
hidden_size = args.hidden_size # 2 # 4
batch_size = args.batch_size #1
output_size = args.output_size  #2
seq_len = args.seq_len  #12 #3  #6 

In [5]:
with g1.as_default():
    ############## Graph construction ################
    data = tf.placeholder(tf.float32, shape=[batch_size, seq_len])
    if args.train_method == "single":
        target = tf.placeholder(tf.float32, shape=[batch_size, output_size])
    else:
        target = tf.placeholder(tf.float32, shape=[seq_len, batch_size, output_size])
        target_list = [tf.squeeze(tf.slice(target, [i,0,0], [1, batch_size, output_size])) for i in range(seq_len)]
    learning_rate = tf.placeholder(tf.float32, shape=[])

    lstm = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
    # initialize the state
    initial_state = state = tf.zeros([batch_size, hidden_size])

    for i in range(seq_len):
        if i > 0: tf.get_variable_scope().reuse_variables()
        with tf.name_scope("in") as scope:
            weights = tf.Variable(tf.truncated_normal([hidden_size, batch_size], stddev=1.0/math.sqrt(float(hidden_size)), name="weights"))
            biases = tf.Variable(tf.truncated_normal([hidden_size], stddev=1.0/math.sqrt(float(hidden_size)), name="biases"))
            mapped = weights *  data[:,i]
            mapped = tf.reshape(mapped, [1, hidden_size]) # 1 x hidden_size
            mapped = mapped + biases
            #mapped = tf.Print(mapped, [mapped], message="this is mapped: ")
        cell_output, state = lstm(mapped, state)
        with tf.name_scope("out") as scope:
            weights = tf.Variable(tf.truncated_normal([hidden_size, output_size], stddev=1.0/math.sqrt(float(hidden_size)), name="weights"))
            biases = tf.Variable(tf.truncated_normal([output_size], stddev=1.0/math.sqrt(float(hidden_size)), name="biases"))
            output = tf.nn.softmax(tf.matmul(cell_output, weights) + biases)  # the size of output should be just [batch_size, 1] right?
            #output = tf.reshape(output, [1,2])
        if args.train_method == "single":                                                    
            pass
        else:
            # target has to be shape=[seq_len * [1,2]]
            loss_per_digit = -tf.reduce_sum(target_list[i]*tf.log(output)) # this should be just 1 by 1 - 1 by 1            
            if args.grad_clip:
                tvars = tf.trainable_variables()
                grads, _ = tf.clip_by_global_norm(tf.gradients(loss_per_digit, tvars), args.max_grad_norm)
                optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
                train_op = optimizer.apply_gradients(zip(grads, tvars))
            else:
                train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    if args.train_method == "single":
        loss = -tf.reduce_sum(target*tf.log(output)) # this should be just 1 by 1 - 1 by 1
        tf.scalar_summary("loss", loss)
        #train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
        if args.grad_clip:
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), args.max_grad_norm)
            #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = optimizer.apply_gradients(zip(grads, tvars))
        else:
            train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # 0.001
    else:
        loss = tf.identity(loss_per_digit)
    
    init_op = tf.initialize_all_variables()
    ############### Graph construction end ##########

In [6]:
with g1.as_default():
    saver = tf.train.Saver(tf.all_variables())

In [7]:
with g1.as_default():
    with tf.Session() as sess:
        #new_saver = tf.train.import_meta_graph('./hidden-uni-vis-test-2016-07-07-09:10:05/my_model-0.meta')
        #print(new_saver.GraphDef())
        saver.restore(sess, './hidden-uni-vis-test-2016-07-07-09:10:05/my_model-0')
        #print(ops.get_default_graph().as_graph_def())
        #graph_def = ops.get_default_graph().as_graph_def()
        
        #var_dict = {}
        #var_name_list = []
        #for var in tf.trainable_variables():
         #   print(var.value().name)
              #print(var.eval())
            # var_dict[var.value().name] = sess.run(var) #tf.constant(var.eval())  # was just var.name before
            #var_name_list.append(var.value().name)

In [19]:
with g1.as_default():
    #graph_def = g1.as_graph_def()
    #for node in graph_def.node:
        #print(node.name)
        #print(node.attr['value'].tensor)
        #if node.name == "in_1/weights":
          #  print("in_1/weights")
            #print(node.attr['value'].tensor)
        #if node.name == "in_2/weights":
          #  print("in_2/weights")
            #print(node.attr['value'].tensor)

        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for var in tf.trainable_variables():
                #print(var.name)
                if var.name == "in_1/Variable:0":
                    print(var.eval())
                if var.name == "in_2/Variable:0":
                    print(var.eval())
            

[[-0.81977516]]
[[-1.45694232]]


In [25]:
g2 = tf.Graph()
with g2.as_default():
    with tf.Session() as sess:
        new_saver = tf.train.import_meta_graph('./hidden-uni-vis-test-2016-07-07-09:10:05/my_model-0.meta')
        #print(new_saver.GraphDef())
        new_saver.restore(sess, './hidden-uni-vis-test-2016-07-07-09:10:05/my_model-0')
        #print(ops.get_default_graph().as_graph_def())
        #graph_def = ops.get_default_graph().as_graph_def()
        
        for var in tf.trainable_variables():
            #print(var.name)
            if var.name == "out_1/Variable_1:0":
                print(var.eval())
            if var.name == "out_2/Variable_1:0":
                print(var.eval())

[-0.97308564 -0.54189807]
[-1.36001205  1.28313386]


In [23]:
with g2.as_default():
    with tf.Session() as sess:
        for var in tf.trainable_variables():
            print(var.name)
            #if var.name == "in_1/Variable:0":
             #   print(var.eval())
            #if var.name == "in_2/Variable:0":
              #  print(var.eval())

in/Variable:0
in/Variable_1:0
BasicRNNCell/Linear/Matrix:0
BasicRNNCell/Linear/Bias:0
out/Variable:0
out/Variable_1:0
in_1/Variable:0
in_1/Variable_1:0
out_1/Variable:0
out_1/Variable_1:0
in_2/Variable:0
in_2/Variable_1:0
out_2/Variable:0
out_2/Variable_1:0


In [27]:
g3 = tf.Graph()
with g3.as_default():
    ############## Graph construction ################
    data = tf.placeholder(tf.float32, shape=[batch_size, seq_len])
    if args.train_method == "single":
        target = tf.placeholder(tf.float32, shape=[batch_size, output_size])
    else:
        target = tf.placeholder(tf.float32, shape=[seq_len, batch_size, output_size])
        target_list = [tf.squeeze(tf.slice(target, [i,0,0], [1, batch_size, output_size])) for i in range(seq_len)]
    learning_rate = tf.placeholder(tf.float32, shape=[])

    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
    # initialize the state
    initial_state = state = tf.zeros([batch_size, hidden_size])

    for i in range(seq_len):
        if i > 0: tf.get_variable_scope().reuse_variables()      
        cell_output, state = rnn_cell(tf.reshape(data[:,i], [1,1]), state)
        output = tf.nn.softmax(cell_output)  # the size of output should be just [batch_size, 1] right?
        
        if args.train_method == "single":                                                    
            pass
        else:
            # target has to be shape=[seq_len * [1,2]]
            loss_per_digit = -tf.reduce_sum(target_list[i]*tf.log(output)) # this should be just 1 by 1 - 1 by 1            
            if args.grad_clip:
                tvars = tf.trainable_variables()
                grads, _ = tf.clip_by_global_norm(tf.gradients(loss_per_digit, tvars), args.max_grad_norm)
                optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
                train_op = optimizer.apply_gradients(zip(grads, tvars))
            else:
                train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    if args.train_method == "single":
        loss = -tf.reduce_sum(target*tf.log(output)) # this should be just 1 by 1 - 1 by 1
        tf.scalar_summary("loss", loss)
        #train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
        if args.grad_clip:
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), args.max_grad_norm)
            #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            train_op = optimizer.apply_gradients(zip(grads, tvars))
        else:
            train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # 0.001
    else:
        loss = tf.identity(loss_per_digit)
    
    init_op = tf.initialize_all_variables()
    ############### Graph construction end ##########

In [55]:
with g3.as_default():
    for var in tf.trainable_variables():
        #print(var.name)
        print(var.name)
        print(var.get_shape())
        print(var.graph)
        

BasicRNNCell/Linear/Matrix:0
(2, 1)
BasicRNNCell/Linear/Bias:0
(1,)
